In [1]:
#Import all required libraries for reading data, analysing and visualizing data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE , SVMSMOTE, BorderlineSMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
import pickle
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Preprocessing

In [4]:
#loading the dataset
df=pd.read_csv('/content/drive/MyDrive/ML_Tugas1/indian_liver_patient.csv')

In [5]:
#dataset showing the top five rows and last five rows
df

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1
...,...,...,...,...,...,...,...,...,...,...,...
578,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
579,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
580,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
581,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1


In [6]:
# replacing missing values with mean
df.Albumin_and_Globulin_Ratio.fillna(df.Albumin_and_Globulin_Ratio.mean(), inplace=True)

In [7]:
#label encoding and scaling
from sklearn.preprocessing import LabelEncoder, RobustScaler
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df.Gender.head()

0    0
1    1
2    1
3    1
4    1
Name: Gender, dtype: int64

In [8]:
#Data Balancing(Resample)
from sklearn.utils import resample
df.Dataset.value_counts()

1    416
2    167
Name: Dataset, dtype: int64

In [9]:
# Split data on majority(1) and minority(2)
minority = df[df.Dataset==2]
majority = df[df.Dataset==1]

print('Minority size:', minority.shape)
print('Majority size:', majority.shape)

Minority size: (167, 11)
Majority size: (416, 11)


In [10]:
# choosing upsample
minority_upsample = resample(minority, replace=True, n_samples=majority.shape[0])
# merge majority with upsampled minority
df = pd.concat([minority_upsample, majority], axis=0)
print('Minority upsampled size:', minority_upsample.shape)

Minority upsampled size: (416, 11)


#1 Feature

Feature Age

In [ ]:
df1 = df.filter(['Age','Dataset'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.16049767 0.13635778 0.10333824 0.12970257 0.13881636 0.14378285
 0.12174106 0.13393092 0.15548778 0.14157438]
Score Time = 
 [0.00339651 0.00177836 0.00176859 0.00178313 0.00176787 0.00186205
 0.00175405 0.00180721 0.00182009 0.00184298]
Test Score = 
 [0.62686567 0.58208955 0.55223881 0.52238806 0.56716418 0.59090909
 0.5        0.43939394 0.53030303 0.60606061]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.64      0.50      0.56       332
           2       0.59      0.73      0.65       333

    accuracy                           0.61       665
   macro avg       0.62      0.61      0.61       665
weighted avg       0.62      0.61      0.61       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.6

Feature Gender

In [ ]:
df1 = df.filter(['Gender','Dataset'])
df1.head()

,Gender,Dataset
355,1,2
291,1,2
189,0,2
56,1,2
534,1,2


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)




Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


In [ ]:


# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.04805255 0.04659343 0.04493785 0.04540467 0.05147076 0.04595017
 0.04455495 0.04817176 0.04451323 0.04498792]
Score Time = 
 [0.00239205 0.00291109 0.00186682 0.00183296 0.00198126 0.00181103
 0.00181127 0.00177836 0.00202942 0.00179553]
Test Score = 
 [0.46268657 0.47761194 0.49253731 0.56716418 0.55223881 0.46969697
 0.43939394 0.28787879 0.45454545 0.46969697]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.51      0.78      0.62       332
           2       0.53      0.25      0.34       333

    accuracy                           0.51       665
   macro avg       0.52      0.51      0.48       665
weighted avg       0.52      0.51      0.48       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.5

Feature Total Bilirubin

In [ ]:
df1 = df.filter(['Total_Bilirubin','Dataset'])
df1.head()

,Total_Bilirubin,Dataset
355,1.4,2
291,0.7,2
189,0.8,2
56,0.8,2
534,1.6,2


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)




Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.05285001 0.05086803 0.05558991 0.05545044 0.05298066 0.05323482
 0.05149937 0.05360889 0.05162072 0.05251861]
Score Time = 
 [0.00159597 0.00154138 0.00227642 0.00154376 0.00157046 0.0015378
 0.00151443 0.00165462 0.00158238 0.00152087]
Test Score = 
 [0.68656716 0.67164179 0.56716418 0.70149254 0.67164179 0.68181818
 0.62121212 0.56060606 0.72727273 0.74242424]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.82      0.42      0.55       332
           2       0.61      0.91      0.73       333

    accuracy                           0.66       665
   macro avg       0.71      0.66      0.64       665
weighted avg       0.71      0.66      0.64       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.81

Feature Direct Bilirubin

In [ ]:
df1 = df.filter(['Direct_Bilirubin','Dataset'])
df1.head()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.04759741 0.04755664 0.05098701 0.04984426 0.04885483 0.04730487
 0.04991341 0.04475522 0.04742289 0.0521698 ]
Score Time = 
 [0.00152087 0.00162196 0.00176167 0.00165844 0.0016036  0.0015173
 0.0014863  0.00148273 0.00484133 0.00156212]
Test Score = 
 [0.70149254 0.68656716 0.58208955 0.71641791 0.70149254 0.68181818
 0.63636364 0.54545455 0.72727273 0.77272727]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.84      0.43      0.57       332
           2       0.62      0.92      0.74       333

    accuracy                           0.68       665
   macro avg       0.73      0.67      0.65       665
weighted avg       0.73      0.68      0.66       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.84

Feature Alkaline Phosphotase

In [ ]:
df1 = df.filter(['Alkaline_Phosphotase','Dataset'])
df1.head()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.07193303 0.06633878 0.06433296 0.07664847 0.06521797 0.07025647
 0.07405591 0.06634307 0.07367039 0.06771207]
Score Time = 
 [0.00165248 0.00163269 0.00170279 0.00188661 0.0017035  0.00165105
 0.00168204 0.00161767 0.00178933 0.00165653]
Test Score = 
 [0.6119403  0.6119403  0.58208955 0.70149254 0.70149254 0.62121212
 0.51515152 0.63636364 0.71212121 0.71212121]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.69      0.53      0.60       332
           2       0.62      0.77      0.69       333

    accuracy                           0.65       665
   macro avg       0.66      0.65      0.64       665
weighted avg       0.66      0.65      0.64       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Alamine_Aminotransferase

In [ ]:
df1 = df.filter(['Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df1.head()


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


,Alamine_Aminotransferase,Dataset
355,13,2
291,32,2
189,32,2
56,26,2
534,88,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.06018019 0.05718827 0.05783272 0.05588794 0.05413628 0.05398202
 0.05265045 0.06308246 0.05533051 0.05902672]
Score Time = 
 [0.00166488 0.00163364 0.00164366 0.0017426  0.00168133 0.0016408
 0.00168657 0.00250554 0.00167155 0.00167322]
Test Score = 
 [0.70149254 0.64179104 0.55223881 0.76119403 0.70149254 0.54545455
 0.59090909 0.45454545 0.66666667 0.6969697 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.71      0.48      0.57       332
           2       0.61      0.81      0.69       333

    accuracy                           0.64       665
   macro avg       0.66      0.64      0.63       665
weighted avg       0.66      0.64      0.63       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.59

Feature Aspartate_Aminotransferase    

In [ ]:
df1 = df.filter(['Aspartate_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df1.head()

Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


,Aspartate_Aminotransferase,Dataset
355,26,2
291,14,2
189,28,2
56,23,2
534,74,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.05388808 0.04726434 0.04635715 0.04808879 0.05363464 0.05506682
 0.04584455 0.0482161  0.04985023 0.04743314]
Score Time = 
 [0.00155711 0.00154567 0.00153852 0.00159907 0.00187945 0.00155258
 0.00151372 0.00152874 0.00159693 0.00163341]
Test Score = 
 [0.70149254 0.67164179 0.62686567 0.71641791 0.71641791 0.60606061
 0.54545455 0.54545455 0.74242424 0.68181818]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.76      0.49      0.60       332
           2       0.63      0.85      0.72       333

    accuracy                           0.67       665
   macro avg       0.69      0.67      0.66       665
weighted avg       0.69      0.67      0.66       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Total_Protiens

In [ ]:
df1 = df.filter(['Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df1.head()


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


,Total_Protiens,Dataset
355,8.0,2
291,7.8,2
189,5.2,2
56,8.0,2
534,8.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.08549237 0.08106041 0.07969975 0.08969831 0.07483196 0.07677889
 0.08434796 0.08852172 0.0771966  0.07397628]
Score Time = 
 [0.00190711 0.00196052 0.00193095 0.00189447 0.0018971  0.00193858
 0.00198507 0.00189638 0.00190043 0.00191069]
Test Score = 
 [0.44776119 0.46268657 0.6119403  0.47761194 0.50746269 0.57575758
 0.45454545 0.43939394 0.59090909 0.62121212]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.56      0.62      0.59       332
           2       0.58      0.52      0.55       333

    accuracy                           0.57       665
   macro avg       0.57      0.57      0.57       665
weighted avg       0.57      0.57      0.57       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.4

Feature Albumin
 

In [ ]:
df1 = df.filter(['Albumin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df1.head()


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


,Albumin,Dataset
355,4.6,2
291,4.2,2
189,2.5,2
56,4.0,2
534,4.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.26794934 0.20971322 0.2581408  0.23172426 0.1877861  0.15846729
 0.15868974 0.14344549 0.20945168 0.21642756]
Score Time = 
 [0.00174809 0.00172591 0.00182986 0.00175333 0.00210571 0.00175881
 0.00174356 0.00182962 0.0017457  0.0018034 ]
Test Score = 
 [0.59701493 0.59701493 0.58208955 0.56716418 0.55223881 0.60606061
 0.59090909 0.62121212 0.54545455 0.60606061]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.59      0.61      0.60       332
           2       0.60      0.58      0.59       333

    accuracy                           0.60       665
   macro avg       0.60      0.60      0.60       665
weighted avg       0.60      0.60      0.60       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.5

Feature Albumin_and_Globulin_Ratio

In [ ]:
df1 = df.filter(['Albumin_and_Globulin_Ratio','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop('Dataset', axis=1), df1['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df1.head()


Train values shape: (665, 1)
Test values shape: (167, 1)
Train target shape: (665,)
Test target shape: (167,)


,Albumin_and_Globulin_Ratio,Dataset
355,1.3,2
291,1.1,2
189,0.9,2
56,1.0,2
534,1.0,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.1229279  0.11718059 0.13371897 0.14758992 0.18222165 0.14676118
 0.11919665 0.12301111 0.11516285 0.13632154]
Score Time = 
 [0.00179553 0.00177193 0.00182724 0.00177813 0.00174046 0.00173545
 0.00177455 0.00190568 0.00177121 0.00180411]
Test Score = 
 [0.59701493 0.64179104 0.59701493 0.6119403  0.59701493 0.56060606
 0.59090909 0.66666667 0.62121212 0.68181818]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.67      0.46      0.54       332
           2       0.59      0.78      0.67       333

    accuracy                           0.62       665
   macro avg       0.63      0.62      0.61       665
weighted avg       0.63      0.62      0.61       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.6

#2 Feature

1 Feature Terbaik yankni dengan hasil : Alkaline_Phosphotase : 0.66 - 0.6655192197360872
Sehingga dijadikan acuan pasangan dengan fitur ke 2

Feature Alkaline_Phosphotase  dan Age

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Age','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()


Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Dataset
355,178,19,2
291,174,60,2
189,218,48,2
56,198,33,2
534,230,39,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.0978775  0.1126709  0.10377669 0.09918404 0.10205817 0.12394595
 0.09405589 0.10297894 0.0855124  0.10642719]
Score Time = 
 [0.0019958  0.00158596 0.0015316  0.00163341 0.00168014 0.00154924
 0.00151229 0.0015018  0.00155449 0.00155902]
Test Score = 
 [0.70149254 0.65671642 0.65671642 0.73134328 0.76119403 0.66666667
 0.5        0.59090909 0.74242424 0.6969697 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.84      0.52      0.65       332
           2       0.66      0.90      0.76       333

    accuracy                           0.71       665
   macro avg       0.75      0.71      0.70       665
weighted avg       0.75      0.71      0.70       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Alkaline_Phosphotase  and Gender

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Gender','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()

Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Gender,Dataset
355,178,1,2
291,174,1,2
189,218,0,2
56,198,1,2
534,230,1,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.07779384 0.06997776 0.07515645 0.06938505 0.07468748 0.07235813
 0.0679183  0.07583594 0.07088041 0.06907082]
Score Time = 
 [0.00174499 0.00272608 0.00164294 0.00170922 0.00159359 0.00164294
 0.00162816 0.00173044 0.00166082 0.00166321]
Test Score = 
 [0.65671642 0.64179104 0.6119403  0.76119403 0.70149254 0.63636364
 0.56060606 0.56060606 0.72727273 0.6969697 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.73      0.52      0.61       332
           2       0.63      0.80      0.71       333

    accuracy                           0.66       665
   macro avg       0.68      0.66      0.66       665
weighted avg       0.68      0.66      0.66       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Alkaline_Phosphotase and Total_Bilirubin

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Total_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()


Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Total_Bilirubin,Dataset
355,178,1.4,2
291,174,0.7,2
189,218,0.8,2
56,198,0.8,2
534,230,1.6,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.0646956  0.08602571 0.06819892 0.07316136 0.06861448 0.0658021
 0.06443024 0.08228374 0.06827426 0.07258177]
Score Time = 
 [0.00151467 0.00153971 0.00154829 0.00155473 0.00202274 0.00152779
 0.00144935 0.00154257 0.002352   0.00161266]
Test Score = 
 [0.67164179 0.65671642 0.64179104 0.73134328 0.76119403 0.71212121
 0.68181818 0.56060606 0.75757576 0.8030303 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.86      0.48      0.62       332
           2       0.64      0.92      0.76       333

    accuracy                           0.70       665
   macro avg       0.75      0.70      0.69       665
weighted avg       0.75      0.70      0.69       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.86

Feature Alkaline_Phosphotase  dan Direct_Bilirubin

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()

Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Direct_Bilirubin,Dataset
355,178,0.8,2
291,174,0.2,2
189,218,0.2,2
56,198,0.2,2
534,230,0.8,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')



Cross Validation Result N=10
Fit Time = 
 [0.06334662 0.06323862 0.07934713 0.12225938 0.06742811 0.06501341
 0.06795645 0.12587094 0.10098672 0.06755304]
Score Time = 
 [0.00152922 0.00151157 0.00147295 0.00150657 0.00155902 0.00156212
 0.00147891 0.00222087 0.00149941 0.00154233]
Test Score = 
 [0.68656716 0.67164179 0.62686567 0.71641791 0.73134328 0.71212121
 0.65151515 0.56060606 0.75757576 0.78787879]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.88      0.46      0.60       332
           2       0.64      0.94      0.76       333

    accuracy                           0.70       665
   macro avg       0.76      0.70      0.68       665
weighted avg       0.76      0.70      0.68       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

Feature Alkaline_Phosphotase  and Alamine_Aminotransferase  

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()

Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Alamine_Aminotransferase,Dataset
355,178,13,2
291,174,32,2
189,218,32,2
56,198,26,2
534,230,88,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.11348963 0.07993031 0.06411886 0.07046437 0.09211493 0.13979697
 0.13754582 0.11174297 0.17090678 0.09639883]
Score Time = 
 [0.00272632 0.00161958 0.00153875 0.00160885 0.00257778 0.00957775
 0.00240564 0.00253487 0.00244069 0.00159454]
Test Score = 
 [0.70149254 0.68656716 0.58208955 0.7761194  0.71641791 0.6969697
 0.59090909 0.63636364 0.74242424 0.8030303 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.76      0.59      0.67       332
           2       0.67      0.82      0.73       333

    accuracy                           0.70       665
   macro avg       0.71      0.70      0.70       665
weighted avg       0.71      0.70      0.70       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.75

FEature Alkaline_Phosphotase dan Aspartate_Aminotransferase 

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Aspartate_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()



Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Aspartate_Aminotransferase,Dataset
355,178,26,2
291,174,14,2
189,218,28,2
56,198,23,2
534,230,74,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.16231036 0.10069752 0.09709024 0.14272785 0.20849991 0.20461965
 0.20297861 0.15003586 0.17811489 0.13016629]
Score Time = 
 [0.00233722 0.00350022 0.00234699 0.00237918 0.01276684 0.0023663
 0.00254297 0.00228691 0.002496   0.00243688]
Test Score = 
 [0.62686567 0.64179104 0.56716418 0.7761194  0.76119403 0.65151515
 0.59090909 0.54545455 0.77272727 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.83      0.49      0.62       332
           2       0.64      0.90      0.75       333

    accuracy                           0.69       665
   macro avg       0.73      0.69      0.68       665
weighted avg       0.73      0.69      0.68       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.78

Feature Alkaline_Phosphotase dan Total Proteins

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()

Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Total_Protiens,Dataset
355,178,8.0,2
291,174,7.8,2
189,218,5.2,2
56,198,8.0,2
534,230,8.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.15323901 0.09946012 0.10101676 0.18336368 0.10160613 0.1053009
 0.16801715 0.20274043 0.179636   0.15147066]
Score Time = 
 [0.00176311 0.00166559 0.00167465 0.00163436 0.00167084 0.00165987
 0.00241542 0.00277591 0.00271511 0.00166464]
Test Score = 
 [0.6119403  0.62686567 0.65671642 0.64179104 0.70149254 0.68181818
 0.54545455 0.56060606 0.6969697  0.74242424]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.81      0.52      0.63       332
           2       0.65      0.88      0.75       333

    accuracy                           0.70       665
   macro avg       0.73      0.70      0.69       665
weighted avg       0.73      0.70      0.69       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.80

Feature Alkaline_Phosphotase dan Albumin 

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Albumin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()

Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Albumin,Dataset
355,178,4.6,2
291,174,4.2,2
189,218,2.5,2
56,198,4.0,2
534,230,4.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.17218328 0.2003572  0.33025837 0.11653352 0.09885669 0.10303497
 0.09869552 0.10893965 0.09418559 0.09273696]
Score Time = 
 [0.00262475 0.00257254 0.00256538 0.00158548 0.00261521 0.00162935
 0.00166559 0.00155115 0.00156641 0.0016036 ]
Test Score = 
 [0.64179104 0.64179104 0.58208955 0.74626866 0.67164179 0.65151515
 0.57575758 0.65151515 0.68181818 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.78      0.53      0.63       332
           2       0.65      0.85      0.73       333

    accuracy                           0.69       665
   macro avg       0.71      0.69      0.68       665
weighted avg       0.71      0.69      0.68       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Alkaline_Phosphotase  dan Albumin_and_Globulin_Ratio

In [ ]:
df2 = df.filter(['Alkaline_Phosphotase','Albumin_and_Globulin_Ratio','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop('Dataset', axis=1), df2['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df2.head()


Train values shape: (665, 2)
Test values shape: (167, 2)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Albumin_and_Globulin_Ratio,Dataset
355,178,1.3,2
291,174,1.1,2
189,218,0.9,2
56,198,1.0,2
534,230,1.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.14942837 0.1659224  0.13927197 0.14966488 0.15947723 0.15096498
 0.12696528 0.08587694 0.08641982 0.10651636]
Score Time = 
 [0.00255537 0.00248313 0.00247502 0.0026257  0.00251341 0.00244355
 0.00162053 0.0015552  0.00167775 0.00263119]
Test Score = 
 [0.67164179 0.64179104 0.53731343 0.74626866 0.70149254 0.63636364
 0.59090909 0.57575758 0.66666667 0.74242424]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.75      0.62      0.68       332
           2       0.67      0.79      0.73       333

    accuracy                           0.70       665
   macro avg       0.71      0.70      0.70       665
weighted avg       0.71      0.70      0.70       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

# 3 Feature 

Hasil Top 2 Feature yakni 
Alkaline_Phosphotase dan Age : 0.68 - 0.6837349397590362


Feature Gender 

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Gender','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Gender,Dataset
355,178,19,1,2
291,174,60,1,2
189,218,48,0,2
56,198,33,1,2
534,230,39,1,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.13147855 0.10679746 0.08947182 0.09637237 0.09340453 0.09703684
 0.10682344 0.10209775 0.09962368 0.11067009]
Score Time = 
 [0.00169325 0.00155091 0.00160885 0.00248051 0.00159097 0.00162244
 0.00151157 0.00157571 0.0015409  0.00170684]
Test Score = 
 [0.71641791 0.62686567 0.65671642 0.65671642 0.74626866 0.68181818
 0.57575758 0.57575758 0.75757576 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.85      0.56      0.68       332
           2       0.67      0.90      0.77       333

    accuracy                           0.73       665
   macro avg       0.76      0.73      0.72       665
weighted avg       0.76      0.73      0.72       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Feature Total_Bilirubin  

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Total_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Total_Bilirubin,Dataset
355,178,19,1.4,2
291,174,60,0.7,2
189,218,48,0.8,2
56,198,33,0.8,2
534,230,39,1.6,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.10179973 0.10235167 0.09410024 0.09160709 0.09740233 0.0995872
 0.09950542 0.11247754 0.10413575 0.10196853]
Score Time = 
 [0.00142884 0.00144362 0.00143433 0.00139904 0.00144005 0.00153565
 0.00146961 0.00143385 0.00155306 0.00149918]
Test Score = 
 [0.74626866 0.7761194  0.74626866 0.74626866 0.79104478 0.8030303
 0.68181818 0.68181818 0.77272727 0.84848485]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.95      0.63      0.76       332
           2       0.72      0.97      0.83       333

    accuracy                           0.80       665
   macro avg       0.84      0.80      0.79       665
weighted avg       0.84      0.80      0.79       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.90 

Feature Direct_Bilirubin

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Direct_Bilirubin,Dataset
355,178,19,0.8,2
291,174,60,0.2,2
189,218,48,0.2,2
56,198,33,0.2,2
534,230,39,0.8,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.10052681 0.09971738 0.09595656 0.09659338 0.10909033 0.10011244
 0.09370542 0.10851479 0.10100102 0.11459041]
Score Time = 
 [0.00139546 0.0014658  0.00144315 0.00140929 0.00140524 0.00139999
 0.00139141 0.00142527 0.00143313 0.00228143]
Test Score = 
 [0.76119403 0.7761194  0.70149254 0.73134328 0.79104478 0.8030303
 0.65151515 0.65151515 0.77272727 0.84848485]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.95      0.61      0.75       332
           2       0.72      0.97      0.82       333

    accuracy                           0.79       665
   macro avg       0.83      0.79      0.79       665
weighted avg       0.83      0.79      0.79       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.90

Feature Alamine_Aminotransferase 

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Alamine_Aminotransferase,Dataset
355,178,19,13,2
291,174,60,32,2
189,218,48,32,2
56,198,33,26,2
534,230,39,88,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.20499277 0.19805527 0.12052989 0.12153316 0.18783402 0.16120529
 0.1647079  0.21181774 0.18104362 0.32309461]
Score Time = 
 [0.00328779 0.00222826 0.00140762 0.00235772 0.00232792 0.00230527
 0.00220776 0.0021863  0.00206852 0.00211978]
Test Score = 
 [0.74626866 0.71641791 0.73134328 0.82089552 0.79104478 0.72727273
 0.60606061 0.6969697  0.77272727 0.83333333]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.89      0.63      0.74       332
           2       0.71      0.92      0.80       333

    accuracy                           0.77       665
   macro avg       0.80      0.77      0.77       665
weighted avg       0.80      0.77      0.77       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Aspartate_Aminotransferase

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Aspartate_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Aspartate_Aminotransferase,Dataset
355,178,19,26,2
291,174,60,14,2
189,218,48,28,2
56,198,33,23,2
534,230,39,74,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.15981317 0.14598918 0.15289402 0.17823887 0.16933274 0.17526007
 0.14735103 0.1733005  0.14484739 0.15204573]
Score Time = 
 [0.00215745 0.00215268 0.00226855 0.0022068  0.00244975 0.00235319
 0.00220108 0.00211811 0.00221324 0.00214696]
Test Score = 
 [0.74626866 0.73134328 0.76119403 0.82089552 0.79104478 0.75757576
 0.56060606 0.62121212 0.78787879 0.8030303 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.92      0.64      0.75       332
           2       0.72      0.94      0.82       333

    accuracy                           0.79       665
   macro avg       0.82      0.79      0.78       665
weighted avg       0.82      0.79      0.78       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Feature Total_Protiens 

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Gender','Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()


Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Gender,Total_Protiens,Dataset
355,178,1,8.0,2
291,174,1,7.8,2
189,218,0,5.2,2
56,198,1,8.0,2
534,230,1,8.0,2


In [ ]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.18816996 0.19505882 0.17758465 0.19398546 0.2174716  0.19527268
 0.18271899 0.31497741 0.18664575 0.19826889]
Score Time = 
 [0.00256205 0.00248885 0.00252962 0.00254893 0.00253868 0.00256753
 0.0025146  0.00254297 0.01115704 0.00256944]
Test Score = 
 [0.74626866 0.67164179 0.70149254 0.59701493 0.68656716 0.68181818
 0.56060606 0.60606061 0.6969697  0.77272727]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.78      0.67      0.72       332
           2       0.71      0.81      0.76       333

    accuracy                           0.74       665
   macro avg       0.75      0.74      0.74       665
weighted avg       0.75      0.74      0.74       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Albumin

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Dataset
355,178,19,4.6,2
291,174,60,4.2,2
189,218,48,2.5,2
56,198,33,4.0,2
534,230,39,4.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.2302556  0.23667526 0.26369882 0.20496583 0.24674606 0.22171211
 0.23757553 0.21465468 0.20458078 0.23321223]
Score Time = 
 [0.00238562 0.00255251 0.00251436 0.00244451 0.00232983 0.00241733
 0.00241971 0.00244117 0.01373649 0.00263047]
Test Score = 
 [0.71641791 0.67164179 0.68656716 0.65671642 0.70149254 0.74242424
 0.63636364 0.60606061 0.71212121 0.81818182]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.90      0.67      0.77       332
           2       0.74      0.92      0.82       333

    accuracy                           0.80       665
   macro avg       0.82      0.80      0.80       665
weighted avg       0.82      0.80      0.80       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Feature Albumin_and_Globulin_Ratio 

In [ ]:
df3 = df.filter(['Alkaline_Phosphotase','Age','Albumin_and_Globulin_Ratio','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()

Train values shape: (665, 3)
Test values shape: (167, 3)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin_and_Globulin_Ratio,Dataset
355,178,19,1.3,2
291,174,60,1.1,2
189,218,48,0.9,2
56,198,33,1.0,2
534,230,39,1.0,2


In [ ]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)


print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.23734665 0.19136262 0.21156478 0.19289446 0.19655108 0.2070334
 0.21635675 0.18670154 0.21162415 0.23852181]
Score Time = 
 [0.00232267 0.00235939 0.00225902 0.00228119 0.00235343 0.00230646
 0.00233793 0.00232553 0.00226808 0.00644755]
Test Score = 
 [0.73134328 0.68656716 0.74626866 0.7761194  0.76119403 0.74242424
 0.72727273 0.71212121 0.71212121 0.8030303 ]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.89      0.74      0.81       332
           2       0.78      0.91      0.84       333

    accuracy                           0.83       665
   macro avg       0.84      0.83      0.83       665
weighted avg       0.84      0.83      0.83       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.75

# 4 Feature

Data Feature Top 3 yakni Alkaline_Phosphotase, Age, dan Albumin dengan hasil : 0.72 - 0.7253298909925416


Feature Gender

In [13]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Gender','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df3.head()


Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Gender,Dataset
377,202,20,3.0,0,2
153,178,52,3.6,1,2
194,209,49,3.0,1,2
377,202,20,3.0,0,2
525,210,53,3.9,0,2


In [12]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.68960738 0.51486659 0.56712222 0.5499649  0.79775381 0.49884486
 0.36190009 0.35899425 0.42378807 0.61884737]
Score Time = 
 [0.00362349 0.00357032 0.01261234 0.00369072 0.01183701 0.00363302
 0.01075315 0.00359845 0.00367856 0.00364494]
Test Score = 
 [0.8358209  0.70149254 0.76119403 0.70149254 0.7761194  0.66666667
 0.71212121 0.65151515 0.77272727 0.71212121]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.93      0.79      0.85       332
           2       0.81      0.94      0.87       333

    accuracy                           0.86       665
   macro avg       0.87      0.86      0.86       665
weighted avg       0.87      0.86      0.86       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Total_Bilirubin

In [16]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Total_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df4.head()


Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Total_Bilirubin,Dataset
377,202,20,3.0,0.6,2
153,178,52,3.6,0.6,2
194,209,49,3.0,2.0,2
377,202,20,3.0,0.6,2
525,210,53,3.9,0.9,2


In [15]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.34201837 0.28453588 0.28405571 0.31576967 0.29023266 0.27133465
 0.29071116 0.45552135 0.53480959 0.35804892]
Score Time = 
 [0.0034833  0.0035429  0.00336099 0.00334358 0.00350285 0.00325775
 0.00348949 0.01046753 0.00339985 0.00358295]
Test Score = 
 [0.86567164 0.80597015 0.71641791 0.73134328 0.80597015 0.71212121
 0.74242424 0.62121212 0.81818182 0.81818182]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.97      0.78      0.87       332
           2       0.82      0.98      0.89       333

    accuracy                           0.88       665
   macro avg       0.89      0.88      0.88       665
weighted avg       0.89      0.88      0.88       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.7

Feature Direct_Bilirubin

In [17]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df4.head()



Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Direct_Bilirubin,Dataset
377,202,20,3.0,0.2,2
153,178,52,3.6,0.1,2
194,209,49,3.0,0.6,2
377,202,20,3.0,0.2,2
525,210,53,3.9,0.2,2


In [18]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.30062604 0.3223021  0.29384208 0.30060339 0.31412601 0.27282763
 0.29032636 0.30230236 0.31060934 0.29216385]
Score Time = 
 [0.00379348 0.00375175 0.00353575 0.00343704 0.00352764 0.00330997
 0.0033772  0.00340605 0.00337791 0.00347066]
Test Score = 
 [0.82089552 0.7761194  0.7761194  0.74626866 0.7761194  0.6969697
 0.74242424 0.60606061 0.8030303  0.81818182]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.97      0.78      0.87       332
           2       0.82      0.98      0.89       333

    accuracy                           0.88       665
   macro avg       0.90      0.88      0.88       665
weighted avg       0.90      0.88      0.88       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.83

Feature Alamine_Aminotransferase 

In [19]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df4.head()

Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Alamine_Aminotransferase,Dataset
377,202,20,3.0,12,2
153,178,52,3.6,26,2
194,209,49,3.0,48,2
377,202,20,3.0,12,2
525,210,53,3.9,35,2


In [20]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.55410743 0.4445951  0.45773649 0.44114375 0.36706519 0.35627294
 0.44274068 0.51517463 0.35354614 0.4929502 ]
Score Time = 
 [0.00364566 0.00350833 0.00354362 0.00343585 0.00430512 0.00339413
 0.01041698 0.01059771 0.00370526 0.00346732]
Test Score = 
 [0.80597015 0.80597015 0.71641791 0.79104478 0.79104478 0.6969697
 0.71212121 0.65151515 0.8030303  0.75757576]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.79      0.87       332
           2       0.82      0.97      0.89       333

    accuracy                           0.88       665
   macro avg       0.89      0.88      0.88       665
weighted avg       0.89      0.88      0.88       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.94

Feature Aspartate_Aminotransferase 

In [28]:
f4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df4.head()

Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Albumin_and_Globulin_Ratio,Dataset
377,202,20,3.0,0.9,2
153,178,52,3.6,1.2,2
194,209,49,3.0,1.1,2
377,202,20,3.0,0.9,2
525,210,53,3.9,0.9,2


In [29]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.15051031 0.15310788 0.16647339 0.14938164 0.16231489 0.16899467
 0.16337585 0.22593498 0.30634809 0.29349899]
Score Time = 
 [0.00209928 0.0021317  0.00218248 0.00217199 0.00289154 0.00220251
 0.00187826 0.00318122 0.00319362 0.0033226 ]
Test Score = 
 [0.79104478 0.7761194  0.80597015 0.86567164 0.8358209  0.75757576
 0.75757576 0.62121212 0.84848485 0.81818182]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.89      0.92       332
           2       0.90      0.96      0.93       333

    accuracy                           0.93       665
   macro avg       0.93      0.93      0.93       665
weighted avg       0.93      0.93      0.93       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

Feature Total_Protiens

In [22]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df4.head()

Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Total_Protiens,Dataset
377,202,20,3.0,6.1,2
153,178,52,3.6,6.5,2
194,209,49,3.0,5.7,2
377,202,20,3.0,6.1,2
525,210,53,3.9,8.0,2


In [23]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.28739405 0.27772355 0.32301497 0.3057487  0.49135542 0.43517828
 0.3620069  0.39385223 0.34657502 0.38112712]
Score Time = 
 [0.00360131 0.00347972 0.00358057 0.00343895 0.00349832 0.00348163
 0.00331545 0.01043487 0.00360799 0.00349355]
Test Score = 
 [0.8358209  0.80597015 0.73134328 0.76119403 0.80597015 0.72727273
 0.77272727 0.66666667 0.8030303  0.87878788]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.88      0.92       332
           2       0.89      0.97      0.93       333

    accuracy                           0.92       665
   macro avg       0.93      0.92      0.92       665
weighted avg       0.93      0.92      0.92       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Featue Albumin_and_Globulin_Ratio 

In [30]:
df4 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Albumin_and_Globulin_Ratio','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df4.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


In [31]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.17045498 0.15684175 0.16607094 0.15656781 0.15771294 0.15934396
 0.18833542 0.1475184  0.15232635 0.16286683]
Score Time = 
 [0.00216651 0.00217652 0.00217032 0.00229859 0.00222278 0.00193429
 0.00184798 0.0019815  0.00195312 0.00198555]
Test Score = 
 [0.79104478 0.7761194  0.80597015 0.86567164 0.8358209  0.75757576
 0.75757576 0.62121212 0.84848485 0.81818182]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.89      0.92       332
           2       0.90      0.96      0.93       333

    accuracy                           0.93       665
   macro avg       0.93      0.93      0.93       665
weighted avg       0.93      0.93      0.93       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

#5 Feature

Hasil Top 4 feature yakni 

*   Alkaline_Phosphotase, Age, dan Albumin, dan Aspartate_Aminotransferase : 0.83 - 0.8269506597819851
*   Alkaline_Phosphotase, Age, dan Albumin, dan Albumin_and_Globulin_Ratio : 0.83 - 0.8269506597819851

Dimna hasil dari kedua feature tersebut sama sehingga memiliki nilai yang sama dan bisa diambil salah satu yakni Aspartate dan Albumin_and_Globulin_Ratio akan di hilangkan


Fitur Gender

In [33]:
df5 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df5.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df5.head()


Train values shape: (665, 5)
Test values shape: (167, 5)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Dataset
377,202,20,3.0,13,0,2
153,178,52,3.6,27,1,2
194,209,49,3.0,32,1,2
377,202,20,3.0,13,0,2
525,210,53,3.9,32,0,2


In [34]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.27668929 0.26840711 0.26130533 0.33460307 0.43359184 0.56091928
 0.41245842 0.39165783 0.26515102 0.45977545]
Score Time = 
 [0.00350785 0.00344419 0.00348544 0.00340247 0.00353718 0.00970721
 0.00336242 0.00340891 0.00337911 0.00388527]
Test Score = 
 [0.8358209  0.70149254 0.79104478 0.74626866 0.76119403 0.65151515
 0.71212121 0.75757576 0.78787879 0.75757576]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.87      0.91       332
           2       0.88      0.96      0.92       333

    accuracy                           0.92       665
   macro avg       0.92      0.92      0.92       665
weighted avg       0.92      0.92      0.92       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

Fitur Total_Bilirubin

In [35]:
df5 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Total_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df5.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df5.head()


Train values shape: (665, 5)
Test values shape: (167, 5)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Total_Bilirubin,Dataset
377,202,20,3.0,13,0.6,2
153,178,52,3.6,27,0.6,2
194,209,49,3.0,32,2.0,2
377,202,20,3.0,13,0.6,2
525,210,53,3.9,32,0.9,2


In [36]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.34353995 0.39599347 0.32695413 0.33101368 0.31577086 0.2670002
 0.28788567 0.32742953 0.24882913 0.2696979 ]
Score Time = 
 [0.0035007  0.00346756 0.00359035 0.00332975 0.00335073 0.00316954
 0.00339103 0.00332141 0.00333333 0.003304  ]
Test Score = 
 [0.8358209  0.8358209  0.76119403 0.80597015 0.8358209  0.72727273
 0.77272727 0.6969697  0.8030303  0.74242424]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.99      0.83      0.90       332
           2       0.85      0.99      0.92       333

    accuracy                           0.91       665
   macro avg       0.92      0.91      0.91       665
weighted avg       0.92      0.91      0.91       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.90

Fitur Direct_Bilirubin

In [37]:
df5 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df5.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df5.head()


Train values shape: (665, 5)
Test values shape: (167, 5)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Direct_Bilirubin,Dataset
377,202,20,3.0,13,0.2,2
153,178,52,3.6,27,0.1,2
194,209,49,3.0,32,0.6,2
377,202,20,3.0,13,0.2,2
525,210,53,3.9,32,0.2,2


In [38]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.33210826 0.29020429 0.30038738 0.29884338 0.27449679 0.29200888
 0.28296661 0.33039808 0.30923605 0.29656029]
Score Time = 
 [0.00343537 0.00345159 0.00573111 0.00369883 0.00350904 0.00369
 0.00324702 0.00347137 0.00339222 0.00348115]
Test Score = 
 [0.85074627 0.85074627 0.74626866 0.82089552 0.85074627 0.72727273
 0.77272727 0.66666667 0.78787879 0.77272727]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.98      0.83      0.90       332
           2       0.85      0.98      0.91       333

    accuracy                           0.91       665
   macro avg       0.92      0.91      0.91       665
weighted avg       0.92      0.91      0.91       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.89  

Fitur Alamine_Aminotransferase 

In [39]:
df5 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df5.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df5.head()


Train values shape: (665, 5)
Test values shape: (167, 5)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Alamine_Aminotransferase,Dataset
377,202,20,3.0,13,12,2
153,178,52,3.6,27,26,2
194,209,49,3.0,32,48,2
377,202,20,3.0,13,12,2
525,210,53,3.9,32,35,2


In [40]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.39974713 0.34480262 0.49783111 0.21144485 0.16364479 0.28680897
 0.26735544 0.31596613 0.3129034  0.26952457]
Score Time = 
 [0.00380063 0.01057673 0.00350761 0.00200295 0.00212336 0.00352526
 0.0034225  0.00334525 0.00334096 0.00338411]
Test Score = 
 [0.82089552 0.76119403 0.70149254 0.8358209  0.79104478 0.71212121
 0.72727273 0.75757576 0.84848485 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.96      0.83      0.89       332
           2       0.85      0.97      0.90       333

    accuracy                           0.90       665
   macro avg       0.90      0.90      0.90       665
weighted avg       0.90      0.90      0.90       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Fitur Total_Proteins

In [41]:
df5 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Total_Proteins','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df5.drop('Dataset', axis=1), df3['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df5.head()

Train values shape: (665, 4)
Test values shape: (167, 4)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Dataset
377,202,20,3.0,13,2
153,178,52,3.6,27,2
194,209,49,3.0,32,2
377,202,20,3.0,13,2
525,210,53,3.9,32,2


In [42]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')



Cross Validation Result N=10
Fit Time = 
 [0.35316038 0.34069324 0.30247688 0.34351206 0.4250946  0.31783533
 0.52569413 0.47167444 0.74874735 0.81465864]
Score Time = 
 [0.00346613 0.00368667 0.00346279 0.00361562 0.00357366 0.0036149
 0.01121116 0.0036695  0.01053596 0.01355672]
Test Score = 
 [0.82089552 0.79104478 0.7761194  0.8358209  0.74626866 0.72727273
 0.71212121 0.72727273 0.74242424 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.95      0.79      0.86       332
           2       0.82      0.96      0.89       333

    accuracy                           0.88       665
   macro avg       0.89      0.88      0.87       665
weighted avg       0.89      0.88      0.87       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.87

#Top 6

Data Feature Top 4 yakni Alkaline_Phosphotase, Age, Albumin, Aspartate_Aminotransferase, dan Gender dengan hasil : 0.82 - 0.8210699942627654

Fitur Total_Bilirubin

In [43]:
df6 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df6.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df6.head()


Train values shape: (665, 6)
Test values shape: (167, 6)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Total_Bilirubin,Dataset
377,202,20,3.0,13,0,0.6,2
153,178,52,3.6,27,1,0.6,2
194,209,49,3.0,32,1,2.0,2
377,202,20,3.0,13,0,0.6,2
525,210,53,3.9,32,0,0.9,2


In [44]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.27514815 0.30213356 0.26290607 0.24753356 0.28062344 0.22963524
 0.26433182 0.24106598 0.25838876 0.24781346]
Score Time = 
 [0.00340486 0.00326109 0.00328112 0.00320458 0.0035789  0.0032959
 0.00324893 0.00340509 0.0033071  0.0034008 ]
Test Score = 
 [0.85074627 0.76119403 0.73134328 0.86567164 0.79104478 0.71212121
 0.81818182 0.77272727 0.83333333 0.71212121]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       1.00      0.92      0.96       332
           2       0.92      1.00      0.96       333

    accuracy                           0.96       665
   macro avg       0.96      0.96      0.96       665
weighted avg       0.96      0.96      0.96       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.92

Fitur Direct_Bilirubin 

In [45]:
df6 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df6.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df6.head()

Train values shape: (665, 6)
Test values shape: (167, 6)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Direct_Bilirubin,Dataset
377,202,20,3.0,13,0,0.2,2
153,178,52,3.6,27,1,0.1,2
194,209,49,3.0,32,1,0.6,2
377,202,20,3.0,13,0,0.2,2
525,210,53,3.9,32,0,0.2,2


In [46]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')


Cross Validation Result N=10
Fit Time = 
 [0.24807501 0.26172519 0.2314322  0.24028683 0.24257922 0.51429439
 0.32573462 0.32247734 0.37796569 0.29151058]
Score Time = 
 [0.00342655 0.00323868 0.0031805  0.00337839 0.00367832 0.00355768
 0.00328612 0.00385594 0.0032506  0.00188828]
Test Score = 
 [0.88059701 0.73134328 0.74626866 0.85074627 0.80597015 0.6969697
 0.78787879 0.72727273 0.83333333 0.72727273]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.99      0.92      0.95       332
           2       0.92      0.99      0.96       333

    accuracy                           0.96       665
   macro avg       0.96      0.96      0.96       665
weighted avg       0.96      0.96      0.96       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.91

Fitur Alamine_Aminotransferase 

In [47]:
df6 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df6.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df6.head()

Train values shape: (665, 6)
Test values shape: (167, 6)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Alamine_Aminotransferase,Dataset
377,202,20,3.0,13,0,12,2
153,178,52,3.6,27,1,26,2
194,209,49,3.0,32,1,48,2
377,202,20,3.0,13,0,12,2
525,210,53,3.9,32,0,35,2


In [48]:

# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.25050426 0.23752713 0.23932815 0.28153062 0.28279066 0.25677776
 0.27496362 0.25972629 0.26489592 0.2598815 ]
Score Time = 
 [0.00338697 0.00331545 0.00335693 0.00538349 0.00343657 0.00370812
 0.00346088 0.0036242  0.00340009 0.00327301]
Test Score = 
 [0.85074627 0.76119403 0.80597015 0.8358209  0.74626866 0.71212121
 0.72727273 0.71212121 0.84848485 0.75757576]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.99      0.89      0.94       332
           2       0.90      0.99      0.95       333

    accuracy                           0.94       665
   macro avg       0.95      0.94      0.94       665
weighted avg       0.95      0.94      0.94       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

Fitur Total Proteins

In [49]:
df6 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df6.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df6.head()

Train values shape: (665, 6)
Test values shape: (167, 6)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Total_Protiens,Dataset
377,202,20,3.0,13,0,6.1,2
153,178,52,3.6,27,1,6.5,2
194,209,49,3.0,32,1,5.7,2
377,202,20,3.0,13,0,6.1,2
525,210,53,3.9,32,0,8.0,2


In [50]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.43418479 0.22728682 0.24587178 0.25100064 0.21602559 0.22412872
 0.28065991 0.22544479 0.22010684 0.23558116]
Score Time = 
 [0.00359297 0.00351501 0.00348735 0.00348091 0.00347424 0.00350642
 0.00825691 0.00344563 0.00347805 0.00359821]
Test Score = 
 [0.92537313 0.79104478 0.89552239 0.8358209  0.8358209  0.8030303
 0.74242424 0.71212121 0.87878788 0.86363636]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       1.00      0.97      0.98       332
           2       0.97      1.00      0.98       333

    accuracy                           0.98       665
   macro avg       0.98      0.98      0.98       665
weighted avg       0.98      0.98      0.98       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.87

#Fitur 7

Feature Top 6 yakni 'Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Protiens', 0.83 - 0.8270223752151463 

Fitur Direct_Bilirubin 

In [51]:
df7 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Bilirubin','Direct_Bilirubin','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df7.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df7.head()

Train values shape: (665, 7)
Test values shape: (167, 7)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Total_Bilirubin,Direct_Bilirubin,Dataset
377,202,20,3.0,13,0,0.6,0.2,2
153,178,52,3.6,27,1,0.6,0.1,2
194,209,49,3.0,32,1,2.0,0.6,2
377,202,20,3.0,13,0,0.6,0.2,2
525,210,53,3.9,32,0,0.9,0.2,2


In [52]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.28853416 0.4981637  0.35425615 0.26227856 0.3015728  0.25323224
 0.27289343 0.26643491 0.23998427 0.22283936]
Score Time = 
 [0.01090693 0.01349378 0.00326753 0.00328398 0.00377083 0.00346231
 0.00345254 0.00352907 0.00322223 0.00318098]
Test Score = 
 [0.85074627 0.7761194  0.7761194  0.88059701 0.80597015 0.74242424
 0.81818182 0.77272727 0.84848485 0.75757576]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.99      0.93      0.96       332
           2       0.94      0.99      0.96       333

    accuracy                           0.96       665
   macro avg       0.96      0.96      0.96       665
weighted avg       0.96      0.96      0.96       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.8

Fitur Alamine_Aminotransferase

In [53]:
df7 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Bilirubin','Alamine_Aminotransferase','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df7.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df7.head()

Train values shape: (665, 7)
Test values shape: (167, 7)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Total_Bilirubin,Alamine_Aminotransferase,Dataset
377,202,20,3.0,13,0,0.6,12,2
153,178,52,3.6,27,1,0.6,26,2
194,209,49,3.0,32,1,2.0,48,2
377,202,20,3.0,13,0,0.6,12,2
525,210,53,3.9,32,0,0.9,35,2


In [54]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.2923758  0.22919822 0.22322035 0.24399996 0.14753842 0.22249532
 0.36650467 0.22220159 0.13151145 0.21878529]
Score Time = 
 [0.00316191 0.00335121 0.00308752 0.00193858 0.00209212 0.01292872
 0.00320339 0.00206137 0.00187588 0.00477552]
Test Score = 
 [0.88059701 0.82089552 0.79104478 0.82089552 0.82089552 0.78787879
 0.81818182 0.78787879 0.84848485 0.71212121]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.98      0.95      0.97       332
           2       0.95      0.98      0.97       333

    accuracy                           0.97       665
   macro avg       0.97      0.97      0.97       665
weighted avg       0.97      0.97      0.97       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9

Fitur Total_Proteins

In [55]:
df7 = df.filter(['Alkaline_Phosphotase','Age','Albumin','Aspartate_Aminotransferase','Gender','Total_Bilirubin','Total_Protiens','Dataset'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df7.drop('Dataset', axis=1), df6['Dataset'], test_size=0.20, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

df7.head()

Train values shape: (665, 7)
Test values shape: (167, 7)
Train target shape: (665,)
Test target shape: (167,)


,Alkaline_Phosphotase,Age,Albumin,Aspartate_Aminotransferase,Gender,Total_Bilirubin,Total_Protiens,Dataset
377,202,20,3.0,13,0,0.6,6.1,2
153,178,52,3.6,27,1,0.6,6.5,2
194,209,49,3.0,32,1,2.0,5.7,2
377,202,20,3.0,13,0,0.6,6.1,2
525,210,53,3.9,32,0,0.9,8.0,2


In [56]:
# Support Vector Machines
model = model = SVC(C=100,gamma=1, kernel="rbf", probability=True)
SVC_CrossVal = cross_validate(model, X_train, y_train, cv=KFold(10))
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print('Cross Validation Result N=10')
print('Fit Time = \n',SVC_CrossVal['fit_time'])
print('Score Time = \n',SVC_CrossVal['score_time'])
print('Test Score = \n',SVC_CrossVal['test_score'])
print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('-------------------------------------------------------')
print('Roc_auc score')
print('-------------------------------------------------------')
auc_rf = roc_auc_score(y_test,y_test_hat)
print(auc_rf)
print('')

Cross Validation Result N=10
Fit Time = 
 [0.22224641 0.21237612 0.23130465 0.23075724 0.23812461 0.21500969
 0.22840762 0.20920491 0.2168262  0.23184586]
Score Time = 
 [0.00374269 0.00359535 0.00361252 0.00373507 0.00376916 0.00364923
 0.00393081 0.00346637 0.00363088 0.00386643]
Test Score = 
 [0.86567164 0.80597015 0.86567164 0.82089552 0.82089552 0.78787879
 0.8030303  0.74242424 0.87878788 0.86363636]
SVC(C=100, gamma=1, probability=True)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       1.00      0.98      0.99       332
           2       0.99      1.00      0.99       333

    accuracy                           0.99       665
   macro avg       0.99      0.99      0.99       665
weighted avg       0.99      0.99      0.99       665

Test performance
-------------------------------------------------------
              precision    recall  f1-score   support

           1       0.9